In [ ]:
from fenics import *
import matplotlib.pyplot as plt
from mshr import *
import numpy as np
import os

In [ ]:
def power_spectrum(field):
    fft_field = np.fft.fft2(field)
    fft_field=np.fft.fftshift(fft_field)
    fft_field=(np.abs(fft_field))
    max_fft = np.where(fft_field == fft_field.max())
    fft_field[max_fft[0][0],max_fft[1][0]] = 0      
    return fft_field

In [ ]:

def plot_field_and_fft(field, field_type, fft_field, t): 
    plt.figure(figsize=(11, 4))
    plt.subplot(1, 2, 1)
    if field_type == 'P':
        plt.imshow(field, cmap = 'seismic' , interpolation = 'bilinear')
        plt.title("$P(t={})$".format(t))
    if field_type == 'N':
        plt.imshow(field, cmap = 'gray' , interpolation = 'bilinear')
        plt.title("$N(t={})$".format(t))
    plt.colorbar() 
    
    plt.subplot(1, 2, 2)
    plt.imshow(fft_field, cmap = 'seismic' , interpolation = 'bilinear')
    if field_type == 'P':
            plt.title("$P_{fft}(t=%s)$" % t)
    if field_type == 'N':
            plt.title("$N_{fft}(t=%s)$" % t)
    plt.colorbar()
    
    plt.tight_layout(pad=-1.0)
#    if field_type == 'P':
#        plt.savefig("fields_P_{}.png".format(t))
#    if field_type == 'N':
#        plt.savefig("fields_N_{}.png".format(t))
    plt.show()

In [ ]:
def matrix_correlation_2D(step):
    abs_n = "matrix_" + "N" + "_dft_" + str(step) + ".txt"
    abs_p = "matrix_" + "P" + "_dft_" + str(step) + ".txt"
    mod_n = np.loadtxt(abs_n, float)
    mod_p = np.loadtxt(abs_p, float)

    n2_inv = np.fft.ifftn(mod_n**2)
    max_fft_n2_inv = np.where(n2_inv == n2_inv.max())
    n2_inv[max_fft_n2_inv[0][0],max_fft_n2_inv[1][0]] = 0   
    
    p2_inv = np.fft.ifftn(mod_p**2)
    max_fft_p2_inv = np.where(p2_inv == p2_inv.max())
    p2_inv[max_fft_p2_inv[0][0],max_fft_p2_inv[1][0]] = 0
    
    np_inv = np.fft.ifftn(mod_n*mod_p)
    max_fft_np_inv = np.where(np_inv == np_inv.max())
    np_inv[max_fft_np_inv[0][0],max_fft_np_inv[1][0]] = 0
 
    n2_inv = np.fft.fftshift(n2_inv)
    p2_inv = np.fft.fftshift(p2_inv)
    np_inv = np.fft.fftshift(np_inv)

    file_abs_n  = np.abs(n2_inv)
    file_abs_p  = np.abs(p2_inv)
    file_abs_np = np.abs(np_inv)

    file_n_real = "matrix_" + "N" + "_" + str(step) + ".txt"
    file_p_real = "matrix_" + "P" + "_" + str(step) + ".txt"

    file_load_n_real = np.loadtxt(file_n_real, float)
    file_load_p_real = np.loadtxt(file_p_real, float)

    mean_n = np.mean(file_load_n_real)
    mean_p = np.mean(file_load_p_real)

    print("<N> = ", mean_n, " \t <P> = ", mean_p)

    corr_nn = file_abs_n - mean_n**2
    corr_pp = file_abs_p - mean_p**2
    corr_np = file_abs_np - mean_n*mean_p
    
    return corr_nn, corr_pp, corr_np

In [ ]:
def plot_correlation_matrix(matrix_1, matrix_2, matrix_3, t): 
    plt.figure(figsize=(20, 3))
    
    plt.subplot(141)
    plt.imshow(matrix_1, cmap = 'seismic' , interpolation = 'bilinear')
    plt.title("$corr\_nn(t={})$".format(t))
    plt.colorbar()
    
    plt.subplot(142)
    plt.imshow(matrix_2, cmap = 'seismic' , interpolation = 'bilinear')
    plt.title("$corr\_pp(t={})$".format(t))
    plt.colorbar()

    plt.subplot(143)
    plt.imshow(matrix_2, cmap = 'seismic' , interpolation = 'bilinear')
    plt.title("$corr\_np(t={})$".format(t))
    plt.colorbar()

    plt.tight_layout(pad=-3.0)
    plt.show()


In [ ]:
T=1000
dt=5

In [ ]:
def polar_mean(step, corr_type):
    matrix_file = "matrix_corr_" + corr_type + "_" + str(t) + ".txt"
    matrix = np.loadtxt(matrix_file, float)
    n = len(matrix)  
    dk = 1.0
    #dk = 1.0*len(matrix)
    #dk = 2.0*np.pi/(len(matrix)*1)
    #print("len_corr = ", dk)
    p = np.zeros(int(n/2))
    for i in range(1,int(n/2)): 
        k = dk*i
        suma = 0.
        dtheta = np.pi/90. # two degrees
        
        for j in range (1,180):
            theta = j * dtheta    
            nx = int( k*np.cos(theta)/dk - n/2)
            ny = int( k*np.sin(theta)/dk - n/2)
            suma += matrix[nx,ny]        
        p[i] = suma/180.
        
    filename = "correlation_" + corr_type + "_" + str(step) + ".txt"         
    np.savetxt(filename, p, delimiter="\n")

In [ ]:
def graph_correlation(step, corr_type):
    file = "correlation_"+corr_type+"_"+str(step)+".txt"
    corr = np.loadtxt(file, float)
    
    plt.plot(corr, "-o")
    plt.title("C_{}(t={})".format(corr_type,step))
    plt.xlabel("r")
    plt.ylabel("C_{}(r)".format(corr_type))

In [ ]:
def graph_correlation_semilog(step, corr_type):

    file = "correlation_"+corr_type+"_"+str(step)+".txt"
    corr = np.loadtxt(file, float)

    plt.semilogy(corr,"-o" )
    plt.title("C_{}(t={})".format(corr_type,step))
    plt.xlabel("r")
    plt.ylabel("C_{}(r)".format(corr_type))

In [ ]:
# Time parameters
dt = 5 # step
T = 1000 # end step

In [ ]:
os.chdir('/home/erick/Desktop/FEniCS/FEniCS-projects/diffution systems/predator-prey/fields')

In [ ]:
t = 0  # zero time
while t < T:
    t += dt
    field_N=np.loadtxt('matrix_N_{}.txt'.format(t),float)
    fft_field = power_spectrum(field_N)
    plot_field_and_fft(field_N, 'N', fft_field, t)
    conc_dft_file = "matrix" + "_N_dft_" + str(t) + ".txt"
    np.savetxt(conc_dft_file, fft_field, delimiter="\t")
    
    field_P=np.loadtxt('matrix_P_{}.txt'.format(t), float)
    fft_field = power_spectrum(field_P)
    plot_field_and_fft(field_P, 'P', fft_field, t)
    conc_dft_file = "matrix" + "_P_dft_" + str(t) + ".txt"
    np.savetxt(conc_dft_file, fft_field, delimiter="\t")


In [ ]:
t = 0  # zero time
while t < T:
    t += dt
    print(t)
    corr_nn, corr_pp, corr_np = matrix_correlation_2D(t)
    plot_correlation_matrix(corr_nn, corr_pp, corr_np, t)
    
    matrix_corr_nn_file = "matrix_corr" + "_nn_" + str(t) + ".txt"
    np.savetxt(matrix_corr_nn_file, corr_nn, delimiter="\t")

    matrix_corr_pp_file = "matrix_corr" + "_pp_" + str(t) + ".txt"
    np.savetxt(matrix_corr_pp_file, corr_pp, delimiter="\t")

    matrix_corr_np_file = "matrix_corr" + "_np_" + str(t) + ".txt"
    np.savetxt(matrix_corr_np_file, corr_np, delimiter="\t")
    

In [ ]:
t = 0  # zero time
while t < T:
    t += dt
    polar_mean(t, "nn")
    graph_correlation(t, "nn")
plt.savefig("correlation_nn.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
t = 0  # zero time
while t < T:
    t += dt
    graph_correlation_semilog(t, "nn")
plt.savefig("correlation_nn_semilog.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
t = 0  # zero time
while t < T:
    t += dt
    polar_mean(t, "pp")
    graph_correlation(t, "pp")
plt.savefig("correlation_pp.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
t = 0  # zero time
while t < T:
    t += dt
    graph_correlation_semilog(t, "pp")
plt.savefig("correlation_pp_semilog.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
t = 0  # zero time
while t < T:
    t += dt
    polar_mean(t, "np")
    graph_correlation(t, "np")
plt.savefig("correlation_np.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
t = 0  # zero time
while t < T:
    t += dt
    graph_correlation_semilog(t, "np")
plt.savefig("correlation_np_semilog.pdf", format="pdf", bbox_inches="tight")
plt.show()